In [1]:
from datetime import datetime
from pydantic.dataclasses import dataclass


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    signup_ts: datetime = None


user = User(id='42', signup_ts='2032-06-21T12:00')
user

User(id=42, name='John Doe', signup_ts=datetime.datetime(2032, 6, 21, 12, 0))

In [3]:
import dataclasses
from pprint import pprint
from typing import List, Optional
from pydantic import Field
from pydantic.dataclasses import dataclass


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    friends: List[int] = dataclasses.field(default_factory=lambda: [0])
    age: Optional[int] = dataclasses.field(
        default=None,
        metadata=dict(title='The age of the user', description='do not lie!')
    )
    height: Optional[int] = Field(None, title='The height in cm', ge=50, le=300)


pprint(user.__pydantic_model__.schema())

{'properties': {'age': {'description': 'do not lie!',
                        'title': 'The age of the user',
                        'type': 'integer'},
                'friends': {'items': {'type': 'integer'},
                            'title': 'Friends',
                            'type': 'array'},
                'height': {'maximum': 300,
                           'minimum': 50,
                           'title': 'The height in cm',
                           'type': 'integer'},
                'id': {'title': 'Id', 'type': 'integer'},
                'name': {'default': 'John Doe',
                         'title': 'Name',
                         'type': 'string'}},
 'required': ['id'],
 'title': 'User',
 'type': 'object'}


# Nested dataclasses

In [4]:
from pydantic import AnyUrl
from pydantic.dataclasses import dataclass


@dataclass
class NavbarButton:
    href: AnyUrl


@dataclass
class Navbar:
    button: NavbarButton


navbar = Navbar(button=('https://example.com',))
navbar

Navbar(button=NavbarButton(href=AnyUrl('https://example.com', scheme='https', host='example.com', tld='com', host_type='domain')))

# Stdlib dataclasses and pydantic dataclasses

## Convert stdlib dataclasses into pydantic dataclasses

In [5]:
import dataclasses
import pydantic
from datetime import datetime
from typing import Optional



@dataclasses.dataclass
class Meta:
    modified_date: Optional[datetime]
    seen_count: int


@dataclasses.dataclass
class File(Meta):
    filename: str


File = pydantic.dataclasses.dataclass(File)

file = File(
    filename=b'thefilename',
    modified_date='2020-01-01T00:00',
    seen_count='7',
)
file

_Pydantic_File_2249165026720(modified_date=datetime.datetime(2020, 1, 1, 0, 0), seen_count=7, filename='thefilename')

In [6]:
try:
    File(
        filename=['not', 'a', 'string'],
        modified_date=None,
        seen_count=3,
    )
except pydantic.ValidationError as e:
    print(e)

1 validation error for File
filename
  str type expected (type=type_error.str)


In [7]:
try:
    File(
        filename=['not', 'a', 'string'],
        modified_date=None,
        seen_count=3,
    )
except pydantic.ValidationError as e:
    print(e)

1 validation error for File
filename
  str type expected (type=type_error.str)


## Inherit from stdlib dataclasses

In [8]:
import dataclasses
import pydantic


@dataclasses.dataclass
class Z:
    z: int


@dataclasses.dataclass
class Y(Z):
    y: int = 0


@pydantic.dataclasses.dataclass
class X(Y):
    x: int = 0


foo = X(x=b'1', y='2', z='3')
foo

_Pydantic_X_2249165030496(z=3, y=2, x=1)

In [9]:
try:
    X(z='pika')
except pydantic.ValidationError as e:
    print(e)

1 validation error for X
z
  value is not a valid integer (type=type_error.integer)


## Use of stdlib dataclasses with BaseModel

In [10]:
import dataclasses
from datetime import datetime
from typing import Optional

from pydantic import BaseModel, ValidationError


@dataclasses.dataclass(frozen=True)
class User:
    name: str


@dataclasses.dataclass
class File:
    filename: str
    last_modification_time: Optional[datetime] = None


class Foo(BaseModel):
    file: File
    user: Optional[User] = None


file = File(
    filename=['not', 'a', 'string'],
    last_modification_time='2020-01-01T00:00',
)  # nothing is validated as expected
file

File(filename=['not', 'a', 'string'], last_modification_time='2020-01-01T00:00')

In [11]:
try:
    Foo(file=file)
except ValidationError as e:
    print(e)

1 validation error for Foo
file -> filename
  str type expected (type=type_error.str)


In [12]:
foo = Foo(file=File(filename='myfile'), user=User(name='pika'))

try:
    foo.user.name = 'bulbi'
except dataclasses.FrozenInstanceError as e:
    print(e)

cannot assign to field 'name'


## Use custom types

In [14]:
import dataclasses
import pydantic


class ArbitraryType:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f'ArbitraryType(value={self.value!r})'


@dataclasses.dataclass
class DC:
    a: ArbitraryType
    b: str

In [15]:
my_dc = DC(a=ArbitraryType(value=3), b='qwe')

try:
    class Model(pydantic.BaseModel):
        dc: DC
        other: str

    Model(dc=my_dc, other='other')
except RuntimeError as e:  # invalid as it is now a pydantic dataclass
    print(e)

no validator found for <class '__main__.ArbitraryType'>, see `arbitrary_types_allowed` in Config


In [17]:
class Model(pydantic.BaseModel):
    dc: DC
    other: str

    class Config:
        arbitrary_types_allowed = True


m = Model(dc=my_dc, other='other')
repr(m)

"Model(dc=_Pydantic_DC_2249166075760(a=ArbitraryType(value=3), b='qwe'), other='other')"